In [ ]:
import pandas as pd
import numpy as np
import joblib
import matplotlib.pyplot as plt
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import train_test_split
from pathlib import Path

# Paths
DATA_DIR = Path('../data')
MODELS_DIR = Path('../models')
ENRICHED_CSV_PATH = DATA_DIR / 'oak_wilt_cluster_enriched.csv'

# Ensure models dir exists
MODELS_DIR.mkdir(exist_ok=True)

In [ ]:
# Load Data
print("Loading data...")
df = pd.read_csv(ENRICHED_CSV_PATH)

# Feature Engineering
print("Feature Engineering...")
df['log_density'] = np.log1p(df['point_density'])
df['log_duration'] = np.log1p(df['duration_days'])

feature_cols = [
    'log_duration',
    'log_density',
    'avg_precip',
    'avg_humidity',
    'avg_wind'
]

# Determine target column
target_col = 'radius_ft' 
if target_col not in df.columns:
    if 'radius_m' in df.columns:
        target_col = 'radius_m'
    else:
        raise ValueError("Could not find radius target column (radius_ft or radius_m)")
        
print(f"Using target column: {target_col}")

X = df[feature_cols]
y = df[target_col]

# Handle NaNs
print("Handling NaNs...")
# Combine X and y to drop rows where any are NaN
data_combined = pd.concat([X, y], axis=1)
initial_len = len(data_combined)
data_combined = data_combined.dropna()
print(f"Dropped {initial_len - len(data_combined)} rows with missing values.")

X = data_combined[feature_cols]
y = data_combined[target_col]

print(f"Final dataset shape: {X.shape}")

In [ ]:
# Split for Conformal Prediction (Train vs Calibration)
# We reserve 20% of the data for calibration (Conformal Prediction)
X_train, X_calib, y_train, y_calib = train_test_split(X, y, test_size=0.2, random_state=42)

print(f"Training Set: {len(X_train)} samples")
print(f"Calibration Set: {len(X_calib)} samples")

In [ ]:
print("Training Gradient Boosting Models...")

# 1. Lower Quantile (10th percentile) - Conservative Estimate
model_lower = GradientBoostingRegressor(loss='quantile', alpha=0.1, n_estimators=100, random_state=42)
model_lower.fit(X_train, y_train)

# 2. Median (50th percentile) - Expected Spread
model_median = GradientBoostingRegressor(loss='quantile', alpha=0.5, n_estimators=100, random_state=42)
model_median.fit(X_train, y_train)

# 3. Upper Quantile (90th percentile) - Severe Case
model_upper = GradientBoostingRegressor(loss='quantile', alpha=0.9, n_estimators=100, random_state=42)
model_upper.fit(X_train, y_train)

print("Models trained successfully.")

In [ ]:
print("Calculating Conformal Prediction Scores (CQR)...")

# Predict on Calibration Set
pred_lower_calib = model_lower.predict(X_calib)
pred_upper_calib = model_upper.predict(X_calib)

# Calculate Non-conformity Scores
# Score = max(lower - y, y - upper)
# If y is inside [lower, upper], score is negative (how far inside)
# If y is outside, score is positive (how far outside)
scores = np.maximum(pred_lower_calib - y_calib, y_calib - pred_upper_calib)

# Calculate q-value for 80% coverage (10th to 90th is 80% interval)
# We want to cover 1 - alpha = 0.8
alpha = 0.2 # 20% allowed error
q_hat = np.quantile(scores, 1 - alpha)

# Correction factor for Conformal Prediction
# We will expand (or contract) the interval by q_hat
print(f"Conformal Correction Factor (q_hat): {q_hat:.4f}")

In [ ]:
# Visualize Calibration Results
plt.figure(figsize=(10, 6))
plt.hist(scores, bins=20, alpha=0.7, color='purple', label='Non-conformity Scores')
plt.axvline(q_hat, color='red', linestyle='--', linewidth=2, label=f'q_hat ({q_hat:.2f})')
plt.title('Calibration Scores Distribution')
plt.xlabel('Score (Positive = Outside Interval)')
plt.ylabel('Frequency')
plt.legend()
plt.grid(True, alpha=0.3)
plt.show()

In [ ]:
# Save Models and q_hat
print("Saving models...")
joblib.dump(model_lower, MODELS_DIR / 'gbr_lower.pkl')
joblib.dump(model_median, MODELS_DIR / 'gbr_median.pkl')
joblib.dump(model_upper, MODELS_DIR / 'gbr_upper.pkl')

# Save q_hat
joblib.dump(q_hat, MODELS_DIR / 'conformal_q.pkl')

print(f"Models saved to {MODELS_DIR}")